In [6]:
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry
from datetime import datetime, timedelta
import json

# Load configuration from a JSON file
def load_config(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

# Load latitude and longitude from config.json
config = load_config("config.json")
latitude = config["latitude"]
longitude = config["longitude"]

# Calculate dynamic dates
end_date = (datetime.today() - timedelta(days=1)).date()  # Today - 1 day
start_date = end_date - timedelta(days=60)  # End date - 2 months

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Fetch pollen data
pollen_params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": [
        "alder_pollen", "birch_pollen", "grass_pollen",
        "mugwort_pollen", "olive_pollen", "ragweed_pollen"
    ],
    "start_date": str(start_date),
    "end_date": str(end_date)
}
pollen_response = openmeteo.weather_api("https://air-quality-api.open-meteo.com/v1/air-quality", params=pollen_params)[0]
pollen_hourly = pollen_response.Hourly()

# Build pollen data dictionary
pollen_data = {
    "time": pd.date_range(
        start=pd.to_datetime(pollen_hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(pollen_hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=pollen_hourly.Interval()),
        inclusive="left"
    )
}
for i, var in enumerate(pollen_params["hourly"]):
    pollen_data[f"{var} (grains/m³)"] = pollen_hourly.Variables(i).ValuesAsNumpy()

pollen_df = pd.DataFrame(pollen_data)

# Fetch weather data
weather_params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": str(start_date),
    "end_date": str(end_date),
    "hourly": [
        "temperature_2m", "relative_humidity_2m", "precipitation", "rain",
        "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high",
        "wind_speed_10m", "soil_temperature_0_to_7cm"
    ]
}
weather_response = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=weather_params)[0]
weather_hourly = weather_response.Hourly()

# Build weather data dictionary
weather_data = {
    "time": pd.date_range(
        start=pd.to_datetime(weather_hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(weather_hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=weather_hourly.Interval()),
        inclusive="left"
    )
}
for i, var in enumerate(weather_params["hourly"]):
    weather_data[f"{var}"] = weather_hourly.Variables(i).ValuesAsNumpy()

weather_df = pd.DataFrame(weather_data)

# Merge pollen and weather data
merged_df = pd.merge(weather_df, pollen_df, on="time", how="inner")

# Compute daily averages
merged_df["date"] = pd.to_datetime(merged_df["time"]).dt.date
daily_data = merged_df.groupby("date").mean().reset_index()

# Save only the daily data to a CSV file
output_file = "daily_data.csv"
daily_data.to_csv(output_file, index=False)

print(f"Daily data saved to {output_file}")


Daily data saved to daily_data.csv


In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import joblib
import os
import json

# Load configuration from JSON
def load_config(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

# File paths for valley boundaries and models
valley_boundaries_file = "Valley_Boundaries.csv"
models_folder = "ValleysModels/"

# Load the valley boundaries
valley_boundaries = pd.read_csv(valley_boundaries_file)
valley_boundaries['geometry'] = valley_boundaries['Boundary Coordinates'].apply(
    lambda x: Polygon(eval(x)) if pd.notnull(x) else None
)
valley_gdf = gpd.GeoDataFrame(valley_boundaries, geometry='geometry')

# Function to determine the valley or closest valley
def get_valley_or_model(lat, lon):
    point = Point(lon, lat)
    for _, row in valley_gdf.iterrows():
        if row['geometry'] and row['geometry'].contains(point):
            valley_name = row['Valley']
            model_file = os.path.join(models_folder, f"{valley_name}_models.pkl")
            return valley_name, model_file
    valley_gdf['distance'] = valley_gdf['geometry'].apply(lambda geom: geom.centroid.distance(point) if geom else float('inf'))
    closest_valley = valley_gdf.loc[valley_gdf['distance'].idxmin()]
    valley_name = closest_valley['Valley']
    model_file = os.path.join(models_folder, f"{valley_name}_models.pkl")
    return valley_name, model_file

# Function to rename columns for consistency
def rename_columns(data):
    column_mapping = {
        "alder_pollen (grains/m³)": "Alder pollen (grains/m³)",
        "birch_pollen (grains/m³)": "Birch pollen (grains/m³)",
        "grass_pollen (grains/m³)": "Grass pollen (grains/m³)",
        "mugwort_pollen (grains/m³)": "Mugwort pollen (grains/m³)",
        "olive_pollen (grains/m³)": "Olive pollen (grains/m³)",
        "ragweed_pollen (grains/m³)": "Ragweed pollen (grains/m³)",
        "precipitation": "precipitation (mm)",
        "temperature_2m": "temperature_2m (°C)"
    }
    return data.rename(columns=column_mapping)

# Function to generate lagged features
def generate_lagged_features(data, pollen_type):
    data[f"{pollen_type}_lag_1"] = data[pollen_type].shift(1)
    data[f"{pollen_type}_lag_2"] = data[pollen_type].shift(2)
    data[f"{pollen_type}_lag_3"] = data[pollen_type].shift(3)
    return data

# Function to add missing features
def add_missing_features(data, required_features):
    for feature in required_features:
        if feature not in data.columns:
            data[feature] = 0  # Default value for missing features
    return data

# Function to load the model
def load_model(model_path):
    if os.path.exists(model_path):
        return joblib.load(model_path)
    else:
        raise FileNotFoundError(f"Model file not found at: {model_path}")

# Function to predict pollen levels
def predict_pollen(model, data, features):
    X = data[features]
    return model.predict(X)

# Main predictive functionality
def main(config_file, historical_data_path):
    # Load configuration
    config = load_config(config_file)
    lat = config["latitude"]
    lon = config["longitude"]
    pollen_types = config["pollen_types"]
    selected_pollen = config["selected_pollen"]

    # Ensure selected pollen type is valid
    if selected_pollen not in pollen_types:
        raise ValueError(f"Invalid pollen type selected: {selected_pollen}. Valid options: {pollen_types}")

    # Determine the valley and model file
    valley_name, model_file = get_valley_or_model(lat, lon)
    print(f"Using model for valley: {valley_name} -> {model_file}")

    # Load historical data
    historical_data = pd.read_csv(historical_data_path)
    historical_data = rename_columns(historical_data)
    historical_data = generate_lagged_features(historical_data, selected_pollen)

    # Load the dictionary of models
    model_dict = load_model(model_file)

    # Select the specific model for the pollen type
    if selected_pollen not in model_dict:
        raise ValueError(f"No model found for pollen type: {selected_pollen}")
    model = model_dict[selected_pollen]

    # Ensure all required features are present
    required_features = model.feature_names_in_
    historical_data = add_missing_features(historical_data, required_features)

    # Preprocess the historical data
    prepared_data = historical_data.dropna()  # Remove rows with NaN after lagging

    # Make predictions for today
    today_data = prepared_data.iloc[-1:]  # Use the last row for today's prediction
    today_prediction = predict_pollen(model, today_data, required_features)

    # Prepare data for tomorrow's prediction
    tomorrow_data = today_data.copy()
    tomorrow_data[selected_pollen] = today_prediction
    tomorrow_data = generate_lagged_features(tomorrow_data, selected_pollen).iloc[-1:]
    tomorrow_prediction = predict_pollen(model, tomorrow_data, required_features)

    # Return predictions
    return {
        "today_prediction": today_prediction[0],
        "tomorrow_prediction": tomorrow_prediction[0]
    }

# FINAL PART: Predict pollen levels for today and tomorrow
if __name__ == "__main__":
    config_file = "config.json"
    historical_data_file = "daily_data.csv"

    try:
        predictions = main(config_file, historical_data_file)
        print(f"Today's Prediction: {predictions['today_prediction']}")
        print(f"Tomorrow's Prediction: {predictions['tomorrow_prediction']}")
    except Exception as e:
        print(f"An error occurred: {e}")


Using model for valley: Valle di Primiero -> ValleysModels/Valle di Primiero_models.pkl
Today's Prediction: 0.0
Tomorrow's Prediction: 0.0


/opt/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
